In [1]:
import os
import sys

In [2]:
xtensions  = 'D:\\Box Sync\\projects\\bio-imaging\\imaris-XTensions\\python-XTensions'
imaris_lib = 'C:\\Program Files\\Bitplane\\Imaris x64 9.2.1\\XT\\python'

sys.path.insert(0,imaris_lib)
sys.path.insert(1,xtensions)

In [3]:
import ImarisLib

In [4]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [5]:
def GetObjectId():
    
    
    vImarisLib = ImarisLib.ImarisLib()
    vServer = vImarisLib.GetServer()
    vNumberOfObjects = vServer.GetNumberOfObjects()
    
    for vIndex in range(vNumberOfObjects):
        vObjectId = vServer.GetObjectID(vIndex)
        return vObjectId; # work with the ID (return first one)
    
    return -1 # invalid id

In [6]:
aImarisId = GetObjectId()
vImarisLib = ImarisLib.ImarisLib()
vImaris = vImarisLib.GetApplication(aImarisId)
vDataSet = vImaris.GetDataSet()

In [7]:
imaris_file = vImaris.GetCurrentFileName()
imaris_dir = os.path.dirname(imaris_file)
imaris_name = os.path.basename(imaris_file)

#### Get Statistics

In [8]:
from cvbi.base_imaris.stats import get_imaris_statistics

In [9]:
object_type = 'surfaces'
object_name = 'CXCL10 cells'
data_stats  = get_imaris_statistics(vImaris=vImaris, object_type=object_type, object_name=object_name)

In [10]:
data_stats.head(2)

names,trackID,objectID,Acceleration,Acceleration X,Acceleration Y,Acceleration Z,Area,BoundingBoxAA Length X,BoundingBoxAA Length Y,BoundingBoxAA Length Z,...,Time Since Track Start,Velocity Angle X,Velocity Angle Y,Velocity Angle Z,Velocity X,Velocity Y,Velocity Z,Volume,time,track_time
0,1000016740,16740,0.000000,0.000000,0.000000,0.000000,963.449524,20.874023,19.880005,21.0,...,0.000000,153.024872,67.184669,76.384239,-0.017980,0.007824,0.004750,1947.886963,1.0,0.000000
1,1000016740,17136,0.000833,-0.000183,0.000535,-0.000612,1078.757935,27.832001,21.867996,21.0,...,77.776001,126.136169,47.750999,116.584534,-0.025097,0.028618,-0.019045,1980.422119,2.0,77.776001


In [143]:
def get_cell_angle(v1,v2):
    
    cosang = np.dot(v1, v2)
    sinang = np.linalg.norm(np.cross(v1, v2))
    angle = np.arctan2(sinang, cosang)
    angle = np.rad2deg(angle)
    
    return(angle)

In [199]:
def get_track_angles(df_in,return_ids=False):
    
    df = df_in.sort_values(['time'])
    coords = df.loc[:,['Position X','Position Y','Position Z']].values
    
    if coords.shape[0]>=3:
        angles = [np.nan]
        coords_previous = coords[:-2,]
        coords_current  = coords[1:-1,]
        coords_next     = coords[2:,]

        direction_current = coords_previous - coords_current
        direction_next = coords_current - coords_next
        
        for i in range(direction_current.shape[0]):
            
            v1 = direction_current[i]
            v2 = direction_next[i]

            angle = get_cell_angle(v1,v2)
            angles += [angle]
        
        angles+=[np.nan]
    else:
        angles = [np.nan for i in range(coords.shape[0])]
        
    output = np.array(angles)
    
    if return_ids:
        objectIDs = df.objectID
        output = pd.DataFrame({'angle':output},index=objectIDs)
        return(output)
    
    else:
        return(output)

In [211]:
data_angles = data_stats.groupby('trackID').apply(lambda df_in:get_track_angles(df_in, return_ids=True))
data_angles.reset_index(inplace=True)
data_stats_out = pd.merge(left=data_stats,right=data_angles,on=['trackID','objectID'])

In [213]:
data_stats_out.head(25)

,trackID,objectID,Acceleration,Acceleration X,Acceleration Y,Acceleration Z,Area,BoundingBoxAA Length X,BoundingBoxAA Length Y,BoundingBoxAA Length Z,...,Velocity Angle X,Velocity Angle Y,Velocity Angle Z,Velocity X,Velocity Y,Velocity Z,Volume,time,track_time,angle
0,1000016740,16740,0.000000,0.000000,0.000000,0.000000,963.449524,20.874023,19.880005,21.0,...,153.024872,67.184669,76.384239,-0.017980,0.007824,0.004750,1947.886963,1.0,0.000000,NaN
1,1000016740,17136,0.000833,-0.000183,0.000535,-0.000612,1078.757935,27.832001,21.867996,21.0,...,126.136169,47.750999,116.584534,-0.025097,0.028618,-0.019045,1980.422119,2.0,77.776001,58.783071
2,1000016740,17476,0.000897,0.000028,-0.000588,0.000677,653.832397,16.897980,11.928009,24.0,...,134.868484,53.000484,111.989708,-0.031117,0.026547,-0.016514,1143.684448,3.0,155.552002,71.679954
3,1000016740,17831,0.000250,-0.000035,0.000109,-0.000222,878.184753,28.825989,12.922012,24.0,...,165.654053,75.809776,87.919922,-0.031372,0.007939,0.001177,1435.555786,4.0,233.328003,32.834023
4,1000016740,18178,0.000297,0.000264,-0.000087,0.000106,777.243652,22.862000,9.940002,24.0,...,157.274658,68.811508,97.848915,-0.022466,0.008805,-0.003325,1283.132812,5.0,311.104004,15.898614
5,1000016740,18467,0.000050,-0.000032,0.000021,0.000031,735.273682,21.868011,10.933990,24.0,...,154.017181,65.358353,82.264221,-0.013465,0.006246,0.002017,1213.067627,6.0,388.880005,7.901827
6,1000016740,18847,0.000090,-0.000017,-0.000031,-0.000082,697.508179,18.885986,10.933990,24.0,...,159.141068,69.138618,89.913521,-0.015393,0.005867,0.000026,1143.992188,7.0,466.656006,23.874195
7,1000016740,19226,0.000235,-0.000172,-0.000159,0.000023,706.581421,17.891998,11.927994,24.0,...,173.133972,93.841888,95.692787,-0.022754,-0.001534,-0.002272,1130.544189,8.0,544.432007,31.682063
8,1000016740,19595,0.000596,0.000445,0.000188,-0.000349,638.045715,17.891998,11.927994,24.0,...,129.097076,91.264198,140.882523,-0.012155,-0.000424,-0.014954,907.308044,9.0,622.208008,100.480116
9,1000016740,19949,0.000947,-0.000607,-0.000104,0.000719,515.273010,14.910004,9.939987,21.0,...,171.085724,81.251114,91.712509,-0.018476,0.002846,-0.000558,788.141541,10.0,699.984009,132.251745
